### Multi-agent systems
- parallel
- sequential
- loop 


Why Multi-Agent Systems?
Types and why

1. **Modular Specialization (Role-Playing)** - By narrowing the scope for each agent, you increase the accuracy of that specific sub-task.

Even when using the same model (e.g., GPT-4o) for every agent, assigning distinct personas and system prompts reduces "distraction." A single prompt trying to be a coder, a security auditor, and a technical writer often suffers from "lost in the middle" context issues.


2. **Iterative Refinement (The Critic Loop)** - "self-correction" produces significantly higher quality than a single-pass prompt. A single agent is often "blind" to its own logical fallacies; a second agent acts as a fresh set of eyes.

Multi-agent systems allow for Adversarial or Collaborative loops. One agent generates an output, and another agent is specifically tasked with finding flaws in it.

3. **Parallelism and Scalability**
In a complex workflow, a multi-agent system can spin up several "worker" agents to perform tasks simultaneously.


4. **Diverse "State" Management** - "clean" context window. Instead of one massive prompt containing all tools and all data, each agent only carries the information relevant to its specific job.

Different agents can maintain different "memory" or "tools."

Agent A might have access to a Python interpreter.

Agent B might have access to a vector database (RAG).

Agent C might have access to a live API.

5. Error Isolation
If a single-agent prompt fails or hallucinates halfway through a 10-step process, the whole run is usually ruined. In a multi-agent system, you can implement checkpoints.

The Benefit: If the "Researcher Agent" fails to find data, the system can catch that error and retry that specific step before the "Writer Agent" ever starts.

In [1]:
from google.adk.agents import Agent, SequentialAgent, ParallelAgent, LoopAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.tools import AgentTool, FunctionTool, google_search
from google.genai import types

In [2]:
# run in vs code only 
from dotenv import load_dotenv
import os

load_dotenv()
GOOGLE_API_KEY=os.getenv("GOOGLE_API_KEY")
if not GOOGLE_API_KEY:
    raise RuntimeError("missing key")


In [ ]:
# do not run in VS code
# Setup for Colab

# set GOOGLE_API_KEY in Colab secrets
# the below retrieves the set key from the Colab
import os
from kaggle_secrets import UserSecretsClient

try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ Gemini API key setup complete.")
except Exception as e:
    print(
        f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}"
    )

# helper function to be used 

# Define helper functions that will be reused throughout the notebook

from IPython.core.display import display, HTML
from jupyter_server.serverapp import list_running_servers


# Gets the proxied URL in the Kaggle Notebooks environment
def get_adk_proxy_url():
    PROXY_HOST = "https://kkb-production.jupyter-proxy.kaggle.net"
    ADK_PORT = "8000"

    servers = list(list_running_servers())
    if not servers:
        raise Exception("No running Jupyter servers found.")

    baseURL = servers[0]["base_url"]

    try:
        path_parts = baseURL.split("/")
        kernel = path_parts[2]
        token = path_parts[3]
    except IndexError:
        raise Exception(f"Could not parse kernel/token from base URL: {baseURL}")

    url_prefix = f"/k/{kernel}/{token}/proxy/proxy/{ADK_PORT}"
    url = f"{PROXY_HOST}{url_prefix}"

    styled_html = f"""
    <div style="padding: 15px; border: 2px solid #f0ad4e; border-radius: 8px; background-color: #fef9f0; margin: 20px 0;">
        <div style="font-family: sans-serif; margin-bottom: 12px; color: #333; font-size: 1.1em;">
            <strong>⚠️ IMPORTANT: Action Required</strong>
        </div>
        <div style="font-family: sans-serif; margin-bottom: 15px; color: #333; line-height: 1.5;">
            The ADK web UI is <strong>not running yet</strong>. You must start it in the next cell.
            <ol style="margin-top: 10px; padding-left: 20px;">
                <li style="margin-bottom: 5px;"><strong>Run the next cell</strong> (the one with <code>!adk web ...</code>) to start the ADK web UI.</li>
                <li style="margin-bottom: 5px;">Wait for that cell to show it is "Running" (it will not "complete").</li>
                <li>Once it's running, <strong>return to this button</strong> and click it to open the UI.</li>
            </ol>
            <em style="font-size: 0.9em; color: #555;">(If you click the button before running the next cell, you will get a 500 error.)</em>
        </div>
        <a href='{url}' target='_blank' style="
            display: inline-block; background-color: #1a73e8; color: white; padding: 10px 20px;
            text-decoration: none; border-radius: 25px; font-family: sans-serif; font-weight: 500;
            box-shadow: 0 2px 5px rgba(0,0,0,0.2); transition: all 0.2s ease;">
            Open ADK Web UI (after running cell below) ↗
        </a>
    </div>
    """

    display(HTML(styled_html))

    return url_prefix


print("✅ Helper functions defined.")


In [3]:
retry_config=types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1, # Initial delay before first retry (in seconds)
    http_status_codes=[429, 500, 503, 504] # Retry on these HTTP errors
)

In [4]:
# Research Agent: Its job is to use the google_search tool and present findings.
research_agent = Agent(
    name="ResearchAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""You are a specialized research agent. Your only job is to use the
    google_search tool to find 2-3 pieces of relevant information on the given topic and present the findings with citations.""",
    tools=[google_search],
    output_key="research_findings",  # The result of this agent will be stored in the session state with this key.
)

print("✅ research_agent created.")

# Summarizer Agent: Its job is to summarize the text it receives.
summarizer_agent = Agent(
    name="SummarizerAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    # The instruction is modified to request a bulleted list for a clear output format.
    instruction="""Read the provided research findings: {research_findings}
Create a concise summary as a bulleted list with 3-5 key points.""",
    output_key="final_summary",
)

print("✅ summarizer_agent created.")

✅ research_agent created.
✅ summarizer_agent created.


In [6]:
# Root Coordinator: Orchestrates the workflow by calling the sub-agents as tools.
root_agent = Agent(
    name="ResearchCoordinator",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    # This instruction tells the root agent HOW to use its tools (which are the other agents).
    instruction="""You are a research coordinator. Your goal is to answer the user's query by orchestrating a workflow.
1. First, you MUST call the `ResearchAgent` tool to find relevant information on the topic provided by the user.
2. Next, after receiving the research findings, you MUST call the `SummarizerAgent` tool to create a concise summary.
3. Finally, present the final summary clearly to the user as your response.""",
    # We wrap the sub-agents in `AgentTool` to make them callable tools for the root agent.
    tools=[AgentTool(research_agent), AgentTool(summarizer_agent)],
)

print("✅ root_agent created.")

✅ root_agent created.


In [7]:
runner = InMemoryRunner(agent=root_agent)
response = await runner.run_debug(
    "What are ways to evaluate agentic AI systems in finance? "
)


 ### Created new session: debug_session_id

User > What are ways to evaluate agentic AI systems in finance? 


ResearchCoordinator > Here are the key ways to evaluate agentic AI systems in finance:

*   **Measure Core Performance:** Utilize metrics for accuracy (e.g., success rate, precision, recall, error rates) and efficiency (e.g., task execution time, time saved for humans, process acceleration).
*   **Quantify Business Impact:** Track strategic ROI and financial metrics such as cost reduction, revenue growth, task automation rate, and cost per resolution.
*   **Assess User Interaction and Trust:** Evaluate user adoption, satisfaction scores, and ensure trust calibration to understand how effectively users engage with the AI.
*   **Employ Advanced and Domain-Specific Metrics:** Consider metrics like hallucination rate, auditability, explainability, compliance adherence, predictive accuracy, risk detection, and cross-agent collaboration.
*   **Leverage Frameworks and Methods:** Employ structured evaluation frameworks (e.g., CLASSic) and methods like synthetic benchmarks, real-task replays, a

In [9]:
import logging

# Set logging to DEBUG to see the internal "handshakes" between agents
logging.basicConfig(level=logging.DEBUG)

In [8]:
# Instead of just awaiting the response, iterate through the events
events = runner.run_debug("What are the latest advancements in quantum computing?")

async for event in events:
    # This captures when a tool (like ResearchAgent) is called
    if event.type == "tool_start":
        print(f"🛠️  Calling Tool: {event.tool_name} with args: {event.tool_args}")
    
    # This captures the output of that tool (the sub-agent's findings)
    elif event.type == "tool_end":
        print(f"✅ Tool {event.tool_name} returned data.")
        # You can inspect event.output to see the raw findings
    
    # This is the final response from the root_agent
    elif event.is_final_response():
        print(f"\n🏁 FINAL SUMMARY:\n{event.content.parts[0].text}")

TypeError: 'async for' requires an object with __aiter__ method, got coroutine